In [19]:
from IPython.display import HTML, display
from matplotlib import pyplot as plt
from datetime import datetime
import simplejson
import urllib
import utm
import gmaps
import pandas as pd
import numpy as np

In [26]:
def calc_euclidean(
    origin: tuple, destiny: tuple
) -> float:
    return np.sqrt(
        (destiny[0]-origin[0])**2 + 
        (destiny[1]-origin[1])**2
    )

In [ ]:
def calc_rectangle(
    origin: tuple, destiny: tuple
) -> float:
    return (
        abs(destiny[0]-origin[0]) + 
        abs(destiny[1]-origin[1])
    )

In [45]:
places = [
    'Badajoz, Teatro López de Ayala',
    'Badajoz, Puente de la Universidad',
    'Badajoz, Centro comercial El Faro',
    'Badajoz, Palacio de Congresos Manuel Rojas',
    'Badajoz, Fuente de Valdepasillas',
    'Badajoz, Puerta del Pilar',
    'Badajoz, La Alcazaba',
    'Badajoz, Plaza Alta',
    'Badajoz, Puerta Palma',
    'Badajoz, Puente Real'
]

In [10]:
#utm.from_latlon(51.2, 7.5)
positions_latlon = []
"""
# 1 - Teatro López de Ayala
https://www.google.com.br/maps/place/Teatro+L%C3%B3pez+de+Ayala/
@38.8762837,-6.9745847,17z/data=!4m13!1m7!3m6!1s0xd16e430fc07a4a1:
0x35a91018d22b30d4!2sTeatro+L%C3%B3pez+de+Ayala!3b1!8m2!3d38.8762837!
4d-6.972396!3m4!1s0xd16e430fc07a4a1:0x35a91018d22b30d4!8m2!
3d38.8762837!4d-6.972396?hl=es-419
"""
positions_latlon.append((38.8762837,-6.9745847))

"""
# 2. Puente de la Universidad
https://www.google.com.br/maps/place/Puente+de+la+Universidad,
+Badajoz,+Espa%C3%B1a/@38.882257,-6.9861811,17z/
data=!3m1!4b1!4m5!3m4!1s0xd16e43efa19b2e5:0x6fd41a9a011acb85!
8m2!3d38.882257!4d-6.9839924?hl=es-419
"""
positions_latlon.append((38.882257, -6.9861811))

"""
# 3. Centro comercial “El Faro”
https://www.google.com.br/maps/place/Centro+Comercial+El+Faro/
@38.884088,-7.0242667,17z/data=!3m1!4b1!4m5!3m4!1s0xd16e3939a74bd99
:0xe713219d4a1bc752!8m2!3d38.884088!4d-7.022078?hl=es-419
"""
positions_latlon.append((38.884088, -7.0242667))

"""
# 4. Palacio de Congresos “Manuel Rojas”,
https://www.google.com.br/maps/place/
Palacio+de+Congresos+Manuel+Rojas+de+Badajoz/
@38.8758768,-6.9704163,17z/data=!3m1!4b1!4m5!3m4!
1s0xd16e43226078b85:0x24be3c99265964d0!8m2!
3d38.8758768!4d-6.9682276?hl=es-419
"""
positions_latlon.append((38.8758768,-6.9704163))

"""
# 5. Fuente de Valdepasillas
https://www.google.com/maps/place/
38%C2%B051'52.1%22N+6%C2%B058'33.1%22W/
@38.864476,-6.9780557,17z/data=!3m1!4b1!4m5!3m4!
1s0x0:0x0!8m2!3d38.864476!4d-6.975867?hl=es
"""
positions_latlon.append((38.864476, -6.9780557))

"""
# 6. Puerta del Pilar
https://www.google.com.br/maps/place/
Puerta+del+Pilar/
@38.8750098,-6.9733397,17z/
data=!3m1!4b1!4m5!3m4!
1s0xd16e431c790860f:0xc8b13668a009fc73!8m2!
3d38.8750098!4d-6.971151?hl=es-419
"""
positions_latlon.append((38.8750098, -6.9733397))

"""
# 7. La Alcazaba
https://www.google.com.br/maps/place/
Alcazaba/@38.8834097,-6.9707085,17z/
data=!3m1!4b1!4m5!3m4!1s0xd16e435eeb32fd5:
0x9ee731b4e09cdf91!8m2!3d38.8834097
!4d-6.9685198?hl=es-419
"""
positions_latlon.append((38.8834097, -6.9707085))

"""
# 8. Plaza Alta
https://www.google.com.br/maps/place/
Plaza+Alta,+Badajoz,+Espa%C3%B1a/
@38.8813045,-6.9704468,17z/
data=!3m1!4b1!4m5!3m4!1s0xd16e43433cc7a4b:
0xddc8a38a60ce0012!8m2!3d38.8813045!
4d-6.9682581?hl=es-419
"""
positions_latlon.append((38.8813045, -6.9704468))

"""
# 9. Puerta Palma
https://www.google.com.br/maps/place/
Puerta+de+Palmas,+06001+Badajoz,+Espa%C3%B1a/
@38.8805376,-6.9777905,17z/
data=!3m1!4b1!4m5!3m4!1s0xd16e439cf4d0167:
0x87dc82633e66385!8m2!3d38.8805183!4d-6.9756006?hl=es-419
"""
positions_latlon.append((38.8805376, -6.9777905))

"""
# 10. Puente Real
https://www.google.com.br/maps/place/
Av.+Puente+Real,+Badajoz,+Espa%C3%B1a/
@38.8781387,-6.9965576,17z/
data=!3m1!4b1!4m5!3m4!1s0xd16e41186d95281:
0xbe96b121a97d9f01!8m2!3d38.8781387!4d-6.9943689?hl=es-419
"""
positions_latlon.append((38.8781387, -6.9965576))

In [14]:
gmaps.heatmap(positions_latlon)

In [ ]:
positions_decimal = [
    utm.from_latlon(*loc, 17)[:2] for loc in positions_latlon
]
positions_decimal

In [43]:
print(utm.from_latlon(38.8762837, -6.9745847, 17))
print(utm.from_latlon(38.8762837, -6.9745847))

(6955440.855920419, 8046631.7989542615, 17, 'S')
(675697.1206430902, 4304997.36541177, 29, 'S')


[(6955440.855920419, 8046631.7989542615),
 (6953483.6520855175, 8045477.002256245),
 (6950166.643573362, 8040406.990685266),
 (6955840.093337603, 8047172.489705317),
 (6957233.071334533, 8045305.911172374),
 (6955761.686237765, 8046710.245389225),
 (6954499.294944567, 8047675.120704978),
 (6954888.155408969, 8047559.294218271),
 (6954444.520803674, 8046500.003951844),
 (6953387.079228595, 8043762.081565264)]

### Distances

In [40]:
matrix_real_distance = pd.DataFrame({
    i: [0]*10 for i in range(10)
})

matrix_euclidean_distance = pd.DataFrame({
    i: [0]*10 for i in range(10)
})

matrix_rectangle_distance = pd.DataFrame({
    i: [0]*10 for i in range(10)
})

In [55]:
url = (
    'http://maps.googleapis.com/maps/api/distancematrix/' +
    'json?origins={0}&destinations={1}&mode=driving&' +
    'language=en-EN&sensor=false'
)
url2 = (
    'https://www.google.com.br/maps/dir/{0}/{1}'
)

for i1, loc1 in enumerate(positions_latlon):
    for i2, loc2 in enumerate(positions_latlon):
        # real distance
        # orig_coord = ','.join(map(str, loc1))
        # dest_coord = ','.join(map(str, loc2))
        orig_coord = urllib.parse.quote_plus(places[i1])
        dest_coord = urllib.parse.quote_plus(places[i2])
        
        _url = url.format(orig_coord,dest_coord)
        print(i1, i2)
        print(_url)
        print(url2.format(orig_coord,dest_coord))
        
        result= simplejson.load(urllib.request.urlopen(_url))
        
        matrix_real_distance.iloc[i1, i2] = (
            result['rows'][0]['elements'][0]['distance']['value']
        )
        
        # euclidean distance
        matrix_euclidean_distance.iloc[i1, i2] = (
            calc_euclidean(loc1, loc2)
        )
        
        # rectangle distance
        matrix_rectangle_distance.iloc[i1, i2] = (
            calc_rectangle(loc1, loc2)
        )

0 0
http://maps.googleapis.com/maps/api/distancematrix/json?origins=Badajoz%2C+Teatro+L%C3%B3pez+de+Ayala&destinations=Badajoz%2C+Teatro+L%C3%B3pez+de+Ayala&mode=driving&language=en-EN&sensor=false
https://www.google.com.br/maps/dir/Badajoz%2C+Teatro+L%C3%B3pez+de+Ayala/Badajoz%2C+Teatro+L%C3%B3pez+de+Ayala
0 1
http://maps.googleapis.com/maps/api/distancematrix/json?origins=Badajoz%2C+Teatro+L%C3%B3pez+de+Ayala&destinations=Badajoz%2C+Puente+de+la+Universidad&mode=driving&language=en-EN&sensor=false
https://www.google.com.br/maps/dir/Badajoz%2C+Teatro+L%C3%B3pez+de+Ayala/Badajoz%2C+Puente+de+la+Universidad
0 2
http://maps.googleapis.com/maps/api/distancematrix/json?origins=Badajoz%2C+Teatro+L%C3%B3pez+de+Ayala&destinations=Badajoz%2C+Centro+comercial+El+Faro&mode=driving&language=en-EN&sensor=false
https://www.google.com.br/maps/dir/Badajoz%2C+Teatro+L%C3%B3pez+de+Ayala/Badajoz%2C+Centro+comercial+El+Faro
0 3
http://maps.googleapis.com/maps/api/distancematrix/json?origins=Badajoz%2C+Te

In [53]:
print('REAL DISTANCE')
display(matrix_real_distance)

print('EUCLIDEAN DISTANCE')
display(matrix_euclidean_distance)

print('RECTANGLE DISTANCE')
display(matrix_rectangle_distance)

REAL DISTANCE


,0,1,2,3,4,5,6,7,8,9
0,0,1690,0,0,0,1125,1181,2938,49153,2717
1,2307,0,2307,2307,2307,1808,1842,3550,50375,2090
2,0,1690,0,0,0,1125,1181,2938,49153,2717
3,0,1690,0,0,0,1125,1181,2938,49153,2717
4,0,1690,0,0,0,1125,1181,2938,49153,2717
5,1713,2417,1713,1713,1713,0,1818,1954,48628,2680
6,1008,1933,1008,1008,1008,2388,0,2165,49491,2767
7,1126,2291,1126,1126,1126,1372,742,0,48612,3126
8,50132,50338,50132,50132,50132,48590,49668,48674,0,51244
9,3764,1841,3764,3764,3764,3650,3300,5007,52216,0


EUCLIDEAN DISTANCE


,0,1,2,3,4,5,6,7,8,9
0,0.000000,0.013044,0.050291,0.004188,0.012307,0.001781,0.008112,0.006506,0.005327,0.022051
1,0.013044,0.000000,0.038130,0.017007,0.019550,0.014745,0.015515,0.015763,0.008565,0.011164
2,0.050291,0.038130,0.000000,0.054473,0.050200,0.051730,0.053562,0.053892,0.046612,0.028341
3,0.004188,0.017007,0.054473,0.000000,0.013724,0.003049,0.007539,0.005428,0.008724,0.026239
4,0.012307,0.019550,0.050200,0.013724,0.000000,0.011541,0.020309,0.018469,0.016064,0.023000
5,0.001781,0.014745,0.051730,0.003049,0.011541,0.000000,0.008802,0.006928,0.007097,0.023428
6,0.008112,0.015515,0.053562,0.007539,0.020309,0.008802,0.000000,0.002121,0.007642,0.026381
7,0.006506,0.015763,0.053892,0.005428,0.018469,0.006928,0.002121,0.000000,0.007384,0.026302
8,0.005327,0.008565,0.046612,0.008724,0.016064,0.007097,0.007642,0.007384,0.000000,0.018920
9,0.022051,0.011164,0.028341,0.026239,0.023000,0.023428,0.026381,0.026302,0.018920,0.000000


RECTANGLE DISTANCE


,0,1,2,3,4,5,6,7,8,9
0,0.000000,0.017570,0.057486,0.004575,0.015279,0.002519,0.011002,0.009159,0.007460,0.023828
1,0.017570,0.000000,0.039917,0.022145,0.025906,0.020089,0.016625,0.016687,0.010110,0.014495
2,0.057486,0.039917,0.000000,0.062062,0.065823,0.060005,0.054236,0.056603,0.050027,0.033658
3,0.004575,0.022145,0.062062,0.000000,0.019040,0.003790,0.007825,0.005458,0.012035,0.028403
4,0.015279,0.025906,0.065823,0.019040,0.000000,0.015250,0.026281,0.024437,0.016327,0.032165
5,0.002519,0.020089,0.060005,0.003790,0.015250,0.000000,0.011031,0.009188,0.009979,0.026347
6,0.011002,0.016625,0.054236,0.007825,0.026281,0.011031,0.000000,0.002367,0.009954,0.031120
7,0.009159,0.016687,0.056603,0.005458,0.024437,0.009188,0.002367,0.000000,0.008111,0.029277
8,0.007460,0.010110,0.050027,0.012035,0.016327,0.009979,0.009954,0.008111,0.000000,0.021166
9,0.023828,0.014495,0.033658,0.028403,0.032165,0.026347,0.031120,0.029277,0.021166,0.000000
